# Generate Training Data

We wish to speed up the process of having a human being name entities that they recognize in a paragraph, and having their positions in the paragraph identified and placed in a syntax usable by spaCy's NER training routine.

## Open PDF file, extract page two and display as sentences

In [1]:
import spacy
import PyPDF2
# import slate3k as slate
nlp = spacy.load('en_core_web_sm')

filename = "BarCvDescLJ11.pdf"

#Open PDF file for reading
pdfFile = open(filename, mode="rb")
pdfReader = PyPDF2.PdfFileReader(pdfFile)

# Select a page to work on
pageNumber = 1

# Get text
OnePage = pdfReader.getPage(pageNumber-1) #0-based count
OnePageText = OnePage.extractText()

# Close PDF file
pdfFile.close()

# Replacement code to use slate which handles spacing within some PDFs better
# with open(filename, 'rb') as f:
#     OnePageText = slate.PDF(f).text()

# Remove newlinesxmx82k-&a. It appears multiple newlines together makes
# Spacy think that is the end of a sentence. The PDF reader reads the text in
# an odd fashion
OnePageText = OnePageText.replace('\n','')

# create a spaCy doc object from the page and break it into sentences
doc = nlp(OnePageText)
l=0
for sent in doc.sents:
     print(l, ": ", sent)
     l = l+1


0 :  1  
1 :  Revised September, 2011    BARLEY CULTIVARS FOR CALIFORNIA  
2 :  Lee Jackson, Extension Small Grain Specialist
3 :  (Retired)  Department of Plant Sciences  University of California, Davis     
4 :  The following are descriptions of barley cultivars evaluated in California from 1981 to 2011.
5 :  The descriptions are based on published cultivar releases and data from the UC Regional Cereal Evaluation Tests conducted each year throughout California.
6 :  Yield performance data for most of the cultivars can be found in University of California, Davis,
7 :  Agronomy Progress Reports (
8 :  No.'s 118, 128, 144, 155, 168, 180, 201,209, 217, 223, 229, 233, 236, 244, 249, 254, 259, 262, 265, 272, 276, 279, 286, 288, 290, 293, 295, 296, 301, 303, 304 for 1980-2011, respectively).
9 :  Reports #262 through 304 also can be seen at http://smallgrains.ucdavis.edu.    
10 :  CURRENT CULTIVARS
11 :  WINTER BARLEY   
12 :  EIGHT-TWELVE  
13 :  Eight-Twelve is a six-rowed winter feed ba

## Read in Per-line named entity file and match entities to sentence positions.

In [2]:
import re
import csv
import pandas as pd

fname = "Data/DavisLJ11/barley_p"+str(pageNumber)+"_ner.txt"
#fname = "Data/UIdaho2019/barley_p"+str(pageNumber-40)+"_ner.txt" # Note p1 = 41 in the document

# Covert the nlp senetence generator into a list of sentences
sentences = list(doc.sents)

# Open the file of manually matched pairs (sentence # <tab> word phrase <tab> named entity)
# e.g.:
#  0      AC Metcalfe     CVAR
#  0      two-rowed       TRAT
#  0      barley          CROP
#  1      Agri-Food Candada   ORG
#  1      1997    DATE
file = open(fname)
reader = csv.reader(file, delimiter='\t', quoting=csv.QUOTE_NONE)
data = list()

for row in reader:
    try:
        (sentIndex, phrase, label) = row
        sent = sentences[int(sentIndex)].string.rstrip()
        
        # find all instances of the 'phrase' in the 'sent'.
        iter = re.finditer(r"\b"+phrase+r"\b", sent)
        indices = [m.start(0) for m in iter]
        
        # check to make sure the phrase the user said was there was indeed found
        if len(indices) == 0:
            raise ValueError
                
        # print out all instances
        for i in indices:
#            print(sentIndex, sent, phrase, "("+str(i), i+len(phrase), "'"+label+"')")
            data.append([sentIndex, sent, phrase, "("+str(i)+", "+str(i+len(phrase))+", '"+label+"')"])
            
    except:
        print("Handle manually: ", row)
        
df = pd.DataFrame(data, columns = ["Index", "Sentence", "Phrase", "MatchInfo"])
print(df)


Handle manually:  ['22', 'Crop Science 32(3):828 (1992)', 'JRNL']
   Index                                           Sentence  \
0     13    Eight-Twelve is a six-rowed winter feed barley.   
1     13    Eight-Twelve is a six-rowed winter feed barley.   
2     13    Eight-Twelve is a six-rowed winter feed barley.   
3     13    Eight-Twelve is a six-rowed winter feed barley.   
4     13    Eight-Twelve is a six-rowed winter feed barley.   
..   ...                                                ...   
76    44  At the time of evaluation it was resistant to ...   
77    44  At the time of evaluation it was resistant to ...   
78    44  At the time of evaluation it was resistant to ...   
79    44  At the time of evaluation it was resistant to ...   
80    44  At the time of evaluation it was resistant to ...   

                    Phrase           MatchInfo  
0             Eight-Twelve     (0, 12, 'CVAR')  
1                six-rowed    (18, 27, 'TRAT')  
2                   winter    

## Create a function to clean up overlapping intervals

In [3]:
import re
coordRegex = re.compile(r'(\d+), (\d+)')

def sortByStart(coords):
    """For use in sort routines, return object with lowest (X,Y) values"""
    # split out coordinates that come in as (5, 7, 'CVAR')
    mo = coordRegex.search(coords)
    return(int(mo.group(1)))

def overlaps(coord1, coord2):
    """Check if coordinates of the form 5, 7, 'CVAR' and 32, 46, 'TRAT' overlap"""
    mo1 = coordRegex.search(coord1)
    mo2 = coordRegex.search(coord2)
    coord1Low = int(mo1.group(1))
    coord1High = int(mo1.group(2))
    coord2Low = int(mo2.group(1))
    coord2High = int(mo2.group(2))
    
    if ((coord1High >= coord2Low) and (coord1Low <= coord2Low) or
        (coord2High >= coord1Low) and (coord2Low <= coord1Low)):
        return True
    else:
        return False

def keepFirst(coord1, coord2):
    """Given overlapping coordinates, return the wider encompassing one."""
    mo1 = coordRegex.search(coord1)
    mo2 = coordRegex.search(coord2)
    coord1Low = int(mo1.group(1))
    coord1High = int(mo1.group(2))
    coord2Low = int(mo2.group(1))
    coord2High = int(mo2.group(2))
 
    if (int(coord1High) - int(coord1Low)) >= (int(coord2High) - int(coord2Low)):
        return True
    else:
        return False

# print("Should be false:", overlaps("(5, 7, 'CVAR')", "(32, 46, 'TRAT')"))
# print("Should be true:", overlaps("(26, 46, 'TRAT')", "(32, 46, 'TRAT')"))
# print("Should be true:", overlaps("(26, 46, 'TRAT')", "(26, 46, 'TRAT')"))
# print("Keeper:", keepFirst("34, 46, 'TRAT'", "34, 46, 'TRAT'"))

def cleanIntervals(inputString=""):
    """order intervals like (5, 7, 'CVAR'), (32, 46, 'TRAT'), (26, 46, 'TRAT') and remove overlapping ones."""
    inputString = inputString.lstrip("(").rstrip(")")
    intervalList = inputString.split("), (")
    intervalList.sort(key = sortByStart)
#    print("Sorted Interval List:", intervalList)

    # Pairwise compare every interval in the list to every other interval to check overlap
    keeperList = [True]*len(intervalList) # Logic array to determine if each interval should be kept
    i=0
    for interval1 in intervalList:
        for interval2 in intervalList:
            if interval1 == interval2:
                if intervalList.index(interval1) != i: # when both are the same we reject the higher one
                    keeperList[i] = False
            else:
                if overlaps(interval1, interval2) and keepFirst(interval1, interval2) == False:
                    keeperList[i] = False
        i = i+1
        
#    print("keeperList:", keeperList)
   
    # Build up the return interval list
    returnStr = "("
    for interval, isKeeper in zip(intervalList, keeperList):
        if isKeeper:
            returnStr = returnStr + interval + "), ("
    return (returnStr.rstrip("), (") + ")")
        
# cleanIntervals("(5, 7, 'CVAR'), (32, 46, 'TRAT'), (5, 9, 'CVAR'), (48, 55, 'ORG'), (26, 46, 'TRAT')")
# cleanIntervals("(0, 8, 'CVAR'), (0, 5, 'CVAR'), (21, 26, 'PLAN'), (32, 37, 'CVAR')")
cleanIntervals("(0, 12, 'CVAR'), (39, 49, 'CVAR'), (39, 49, 'CVAR'), (71, 77, 'CVAR'), (71, 77, 'CVAR'), (92, 113, 'TRAT'), (140, 150, 'CVAR'), (140, 150, 'CVAR'), (181, 187, 'CVAR'), (181, 187, 'CVAR')")


"(0, 12, 'CVAR'), (39, 49, 'CVAR'), (71, 77, 'CVAR'), (92, 113, 'TRAT'), (140, 150, 'CVAR'), (181, 187, 'CVAR')"

## Aggregate all matches for each sentence on a single line and output in spaCy training format

In [4]:
# use Pandas dataframes to aggregate all entity matches together for a single sentence
agg_rules = {'Sentence': 'first', 'Phrase': 'first', 'MatchInfo': lambda x: ', '.join(x)}
res = df.groupby('Index').agg(agg_rules)
#print(res)

# Now format it just like what is needed for the spaCy training module: 
# E.g.:
# ('Eight-Twelve is a six-rowed winter feed barley', {'entities': [(0, 12, 'CVAR'), (18, 27, 'TRAT'), (28, 39, 'TRAT'),(40, 46, 'CROP')]}),
records = res.to_dict('records')
print("TRAIN_DATA = [")
maxr = len(records)
for i in range(0,maxr):
    print("    ('"+records[i]['Sentence']+"', {'entities': ["+cleanIntervals(records[i]['MatchInfo'])+"]})", end='')
    if (i == maxr-1):
        print()
    else:
        print(",")

print("]")

TRAIN_DATA = [
    ('Eight-Twelve is a six-rowed winter feed barley.', {'entities': [(0, 12, 'CVAR'), (18, 27, 'TRAT'), (28, 34, 'TRAT'), (35, 39, 'CVAR'), (40, 46, 'CROP')]}),
    ('It was released by the USDA-ARS and the Idaho AES in 1991.', {'entities': [(23, 31, 'ORG'), (40, 49, 'ORG'), (53, 57, 'DATE')]}),
    ('It was selected from the cross Steveland/Luther//Wintermalt.', {'entities': [(31, 59, 'PED')]}),
    ('Its experimental designation was 79Ab812.', {'entities': [(33, 40, 'ALAS')]}),
    ('It has rough awns, midseason maturity, good winter hardiness, and is medium height with intermediate lodging resistance.', {'entities': [(13, 17, 'PLAN'), (29, 37, 'TRAT'), (44, 60, 'TRAT'), (76, 82, 'TRAT'), (101, 119, 'TRAT')]}),
    ('Spikes are short and dense.', {'entities': [(0, 6, 'PLAN')]}),
    ('Kernels have white aleurone and short rachilla hairs.', {'entities': [(0, 7, 'PLAN'), (19, 27, 'PLAN'), (38, 52, 'PLAN')]}),
    ('It is susceptible to stripe rust, scald, and snow mold,

In my current process, I am writing the above content to a file e.g., `Data/DavisLJ11/barley_p5_td.py`, adding any manual corrections (usually PED and JRNL entries) and then running the script `python3 py2json.py --doc 'BarCvDescLJ11.pdf' --url 'https://smallgrains.ucdavis.edu/cereal_files/BarCvDescLJ11.pdf' --chunk 5 Data/DavisLJ11/barley_p5_td.py Data/DavisLJ11/barley_p5_td.json` to create the JSON file for Training.